In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import re 
from utils import * 
import itertools 
from tqdm import tqdm 
import warnings 
import subprocess
import os 
from src.ggkbase import * 
import glob

%load_ext autoreload
%autoreload 2

In [3]:
new_genome_id_map = dict()
new_genome_id_map['SR-VP_05_06_2024_N_bottom.metabat2.2860'] = 'SR-VP_05_06_2024_N_bottom_DPANN-maybe_43_7'
new_genome_id_map['SR-VP_05_06_2024_N_bottom.metabat2.1582'] = 'SR-VP_05_06_2024_N_bottom_Euryarchaeota_66_35'
new_genome_id_map['SR-VP_05_06_2024_N_bottom.metabat2.223'] = 'SR-VP_05_06_2024_N_bottom_DPANN_Archaea_34_18'
new_genome_id_map['SR-VP_05_06_2024_N_top.vamb.17615'] = 'SR-VP_05_06_2024_N_top_Candidatus_Methanoperedens_44_7'
new_genome_id_map['SR-VP_05_06_2024_N_top.metabat2.305'] = 'SR-VP_05_06_2024_N_top_Euryarchaeota_54_20'

In [4]:
metadata_df = load_metadata()
metadata_df = metadata_df[metadata_df.taxonomy.str.contains('Archaea')].copy() # Include only the archaea. 
print('Num. Archaeal MAGs:', len(np.unique(metadata_df.index)))

Num. Archaeal MAGs: 71


In [7]:
sample = 'top'

contigs_path = f'../data/contigs/n_{sample}_contigs.fa'
scaffold_to_bin_path = f'../data/n_{sample}_scaffold_to_bin.tsv'

metadata_df = load_metadata()
metadata_df = metadata_df[(metadata_df.index.str.contains(f'N_{sample}'))].copy()
metadata_df = metadata_df[metadata_df.taxonomy.str.contains('Archaea')].copy() # Include only the archaea. 
metadata_df.index = [new_genome_id_map.get(genome_id, genome_id) for genome_id in metadata_df.index]

for genome_id in tqdm(np.unique(metadata_df.index), desc='Retrieving specified genomes from the contig file...'):
    contigs_path_ = contigs_remove_unknown(path=contigs_path)
    scaffold_to_bin_path_ = scaffold_to_bin_remove_unknown(path=scaffold_to_bin_path)
    contigs_get_genome(genome_id, contigs_path=contigs_path_, scaffold_to_bin_path=scaffold_to_bin_path_)

# genome_id = 'SR-VP_05_06_2024_N_bottom.metabat2.4162'
# get_scaffold_ids(genome_id)

Retrieving specified genomes from the contig file...: 100%|██████████| 28/28 [00:23<00:00,  1.17it/s]


In [8]:

def run_prodigal(fn_dir:str='../data/genomes/', fa_dir='../data/proteins/'):
    for path in tqdm(glob.glob(os.path.join(fn_dir, '*')), desc='run_prodigal'):
        fa_path = re.sub('.fn|.fna', '.fa', os.path.basename(path))
        fa_path = os.path.join(fa_dir, fa_path)
        try:
            if not os.path.exists(fa_path):
                subprocess.run(f'prodigal -i {path} -a {fa_path}', shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except:
            print(f'run_prodigal: Could not run Prodigal on {path}')

run_prodigal()


run_prodigal:   0%|          | 0/48 [00:00<?, ?it/s]

run_prodigal: 100%|██████████| 48/48 [00:00<00:00, 3380.40it/s]

run_prodigal: Could not run Prodigal on ../data/genomes/SR-VP_05_06_2024_N_top_maxbin2_40.fn
run_prodigal: Could not run Prodigal on ../data/genomes/SR-VP_05_06_2024_N_top_maxbin2_12.fn
